## This workbook automates the download of monthly option pricing zip files from Barchartacs.

1. It will use Selenium to login to the site: 
```
ACS_HOME_PAGE = 'http://acs.barchart.com/mri/mripag.htm' 
ACS_LOGIN_INFO = './temp_folder/acs_login.txt'
acs_text = open(ACS_LOGIN_INFO,'r').read()
ACS_USERNAME = acs_text.split(',')[0]
ACS_PASSWORD = acs_text.split(',')[1]
```
2. It will scrape the urls of the zip files, and then download those zipfiles into a folder which is specified using:
```
ZIP_FOLDER_PARENT = open('./temp_folder/zip_folder_parent.txt','r').read()
```

3. You can manually upload one or several months after manuall logging in to Barchartacs at http://acs.barchart.com/mri/mriopt.htm.  Navigate down the page to the table **Monthly Options Zip Files** and then click on a CSV file that you want to download

In [ ]:
import sel_scrape as sc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import requests
from requests.auth import HTTPBasicAuth
import os
import time
import traceback
from tqdm import tqdm,tqdm_notebook
import numpy as np

In [ ]:
MMM_LIST = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
BEGIN_YY = 10
END_YY = 19
YY_LIST = list(np.arange(BEGIN_YY,END_YY+1))
MMMYY_LIST = [mmm + str(yy) for mmm in MMM_LIST for yy in YY_LIST]
MMMYY_LIST

In [ ]:
# import importlib
# importlib.reload(sc)

In [ ]:
ACS_HOME_PAGE = 'http://acs.barchart.com/mri/mripag.htm' 
ACS_FUTURES_PAGE = 'http://acs.barchart.com/mri/mridta.htm'
ACS_OPTIONS_PAGE = 'http://acs.barchart.com/mri/mriopt.htm'
ACS_LOGIN_INFO = './temp_folder/acs_login.txt'
acs_text = open(ACS_LOGIN_INFO,'r').read()
ACS_USERNAME = acs_text.split(',')[0]
ACS_PASSWORD = acs_text.split(',')[1]
ZIP_FOLDER_PARENT = open('./temp_folder/zip_folder_parent.txt','r').read()

In [ ]:
if not os.path.isdir(ZIP_FOLDER_PARENT):
    print(f'making parent folder {ZIP_FOLDER_PARENT}')
    os.mkdir(ZIP_FOLDER_PARENT)
else:
    print(f'parent folder {ZIP_FOLDER_PARENT} already exists')

In [ ]:
sela = sc.SelScrape(headless=False)
sela.goto(ACS_HOME_PAGE)
time.sleep(1)
wait(sela.driver, 5).until(EC.alert_is_present())
alert = sela.driver.switch_to_alert()
alert.send_keys(f'{ACS_USERNAME}{Keys.TAB}{ACS_PASSWORD}')
time.sleep(3)
alert.accept()


In [ ]:
sela.goto(ACS_OPTIONS_PAGE)
# eod_files_xpath = "//a[@href='mrifile.htm']"
# eod_elem = sela.findxpath(eod_files_xpath)['value'][0]
# sela.goto(eod_elem.get_attribute('href'))
# opt_eod_xpath = "//a[@href='mriopt.htm']"
# opt_eod_elem = sela.findxpath(opt_eod_xpath)['value'][0]
# sela.goto(opt_eod_elem.get_attribute('href'))


In [ ]:
monthly_csv_files_xpath = "//a[contains(@href,'data/opt/opv')]"
mcsv_elements = sela.findxpath(monthly_csv_files_xpath)['value']
mcsv_hrefs_all = []
for mcsv in mcsv_elements:
    mcsv_hrefs_all.append(mcsv.get_attribute('href'))
mcsv_hrefs_all

In [ ]:
def is_valid_yyymm(h):
    return any([m in h for m in MMMYY_LIST])
mcsv_hrefs = [h for h in mcsv_hrefs_all if is_valid_yyymm(h) ] 
mcsv_hrefs

In [ ]:
options_parent = ZIP_FOLDER_PARENT+'/options'
if not os.path.isdir(options_parent):
    print(f'making options folder {options_parent}')
    os.mkdir(options_parent)
else:
    print(f'options folder {options_parent} already exists')
hrefs_to_unzip = []
paths_to_unzip_to = []
for mcsv_href in mcsv_hrefs:
    zip_file_name = mcsv_href.split('/')[-1]
    folder_name = zip_file_name.replace('.zip','')
    path_to_zip_folder = f'{options_parent}/{folder_name}'
    if not os.path.isdir(path_to_zip_folder):
        print(f'making {path_to_zip_folder}')
        os.mkdir(path_to_zip_folder)
    path_to_zip_file = f'{path_to_zip_folder}/{zip_file_name}'
    if not os.path.isfile(path_to_zip_file):
        hrefs_to_unzip.append(mcsv_href)
        paths_to_unzip_to.append(path_to_zip_file)

successful_downloads = []
for i in tqdm_notebook(range(len(hrefs_to_unzip))):
    url = hrefs_to_unzip[i]
    path_to_zip_file = paths_to_unzip_to[i]
    try:    
        r=requests.get(url, auth=HTTPBasicAuth(ACS_USERNAME, ACS_PASSWORD))
        p = paths_to_unzip_to[i]
        with open(p, 'wb') as f:
            f.write(r.content)
        successful_downloads.append(path_to_zip_file)
    except Exception as e:
        traceback.print_exc()
        

        